### Notation:
Since objects are considered morphisms (an object $X$ is identified with its identity morphism $\text{id}_X$), we write `X, Y, Z, ...` for objects, `f, g, h, ...` for morphisms that are not objects, and `x, y, z, ...` for morphisms for which itis unknown whether they are objects / identity morphisms.

Objects which are categories are denoted by `C, D, E, ...`.

Functors are denoted by `F, G, H, ...`

In [1]:
class Morphism:
    
    def __init__(self, category, domain, codomain):
        self.category = category
        self.domain = domain
        self.codomain = codomain

In [2]:
class Object(Morphism):

    def __init__(self, category = None):
        # Object() creates the category of categories (whose category is itself)
        if not category:
            category = self
        
        # An object is identified with its identity morphism
        Morphism.__init__(self, category, self, self)

In [3]:
# Do not actually need this. A Morphism f is a Functor precisely if f.category == Cat
# Actually have a representation for opposite categories, I guess..

# class Functor(Morphism):
    
#     def __init__(self, category, domain, codomain):
#         Morphism.__init__(self, category, domain, codomain)

In [4]:
class Representation:
        
    def __init__(self):
        self.ptr = None
        
    def assign(self, ptr):
        self.ptr = ptr
    
#     def replace(self, x, y): # TODO: actually, maybe just have a method 'contains'/'depends_on' or something, and then if this is the case, just recreate the representation, because it might reduce to something else!
#         if self.ptr == x: # TODO: 'is'?
#             self.ptr = y

class Repr_Symbol(Representation):
    
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        return self.name == other.name

class Repr_Composition(Representation):
    
    def __init__(self, f_list):        
        self.f_list = f_list
    
    def __eq__(self, other):
        return self.f_list == other.f_list
    
class Repr_And(Representation):
    
    def __init__(self, P, Q):
        self.P = P
        self.Q = Q
        
    def __eq__(self, other):
        return self.P == other.P and self.Q == other.Q
        
class Repr_Or(Representation):
    
    def __init__(self, P, Q):
        self.P = P
        self.Q = Q
        
    def __eq__(self, other):
        return self.P == other.P and self.Q == other.Q
        
class Repr_Implies(Representation):
    
    def __init__(self, P, Q):
        self.P = P
        self.Q = Q
        
    def __eq__(self, other):
        return self.P == other.P and self.Q == other.Q
        
class Repr_Not(Representation):
    
    def __init__(self, P):
        self.P = P
        
    def __eq__(self, other):
        return self.P == other.P
        
class Repr_Op(Representation):
    
    def __init__(self, C):
        self.C = C
        
    def __eq__(self, other):
        return self.C == other.C
        
class Repr_Functor(Representation):
    
    def __init__(self, F, x):
        self.F = F
        self.x = x
        
    def __eq__(self, other):
        return self.F == other.F and self.x == other.x

class Repr_Property(Representation):
    
    def __init__(self, prop, data):
        self.prop = prop
        self.data = data
        
    def __eq__(self, other):
        return self.prop == other.prop and self.data == other.data
        

In [18]:
class Diagram:
    
    def __init__(self):
        self.data = []
        self.conditions = []
        
        self.morphisms = []
        self.representations = []
        
    def add_data(self, X):
        self.data.append(X)
        self.morphisms.append(X)
            
    def add_condition(self, C):
        self.conditions.append(C)
    
    def add_morphism(self, x):
        self.morphisms.append(x)
    
    def add_representation(self, rep):
        self.representations.append(rep)
    
    def find_representation(self, rep):
        for r in self.representations:
            if type(r) == type(rep) and r == rep:
                return r.ptr
                
        return None
    
    def clean(self):
        # Remove all representations and morphisms that are unnecessary (whatever that means)
        pass
    
    
    
    # --- Non-essential methods ---
    
    def str_x(self, x):
        for r in self.representations:
            if r.ptr == x:
                if isinstance(r, Repr_Symbol):
                    return r.name
                if isinstance(r, Repr_Functor):
                    return '{}({})'.format(self.str_x(r.F), self.str_x(r.x))
                if isinstance(r, Repr_Property):
                    return '{}({})'.format(r.prop.name, ', '.join([ self.str_x(y) for y in r.data ]))
                if isinstance(r, Repr_Composition):
                    return '.'.join([ self.str_x(f) for f in r.f_list ])

        return '?'
    
    def __str__(self):
        return '{{ {} }}'.format(', '.join(self.str_x(x) for x in self.morphisms))

In [6]:
class Property:
    
    def __init__(self, name, diagram):
        self.name = name
        self.diagram = diagram

In [7]:
class Theorem:
    
    pass

In [8]:
class Instruction:
    
    pass

In [9]:
class Condition:
    
    pass

In [10]:
class Factory:
    
    def __init__(self, diagram):
        self.diagram = diagram
    
    def create_object(self, name, C):
        # C must be a category
        if(C.category != Cat):
            raise Exception('That is not a category!')
        
        # Create representation
        rep = Repr_Symbol(name)
        
        # If the representation already exists in the diagram, raise an exception
        if self.diagram.find_representation(rep):
            raise Exception('Name \'{}\' is already used!'.format(name))
        
        # Construct new object
        X = Object(C)
        
        # Assign to representation, and add to the diagram
        rep.assign(X)
        self.diagram.add_morphism(X)
        self.diagram.add_representation(rep)
        
        return X
    
    def create_morphism(self, name, X, Y):
        # The categories of X and Y must be equal
        if X.category != Y.category:
            raise Exception('Cannot construct a morphism between objects of different categories!')

        # Create representation
        rep = Repr_Symbol(name)
        
        # If the representation already exists in the diagram, raise an exception
        if self.diagram.find_representation(rep):
            raise Exception('Name \'{}\' is already used!'.format(name))
        
        # Construct new morphism
        f = Morphism(X.category, X, Y)
        
        # Assign to representation, and add to the diagram
        rep.assign(f)
        self.diagram.add_morphism(f)
        self.diagram.add_representation(rep)
        
        return f
        
    def create_composition(self, f_list):
        # There must be at least one morphism
        if not f_list:
            raise Exception('Composition requires morphisms!')

        # Obtain domain / codomain
        X, Y = f_list[-1].domain, f_list[0].codomain
        
        # All morphisms must connect
        n = len(f_list)
        for i in range(n - 1):
            if f_list[i].domain != f_list[i + 1].codomain:
                raise Exception('Morphisms do not connect well!')
            
        # Remove all identity morphisms from the list
        f_list = [ f for f in f_list if not isinstance(f, Object) ]
        
        # If the list is empty now, then the result would have been id(X) = id(Y)
        n = len(f_list)
        if n == 0:
            return X
        
        # If there is only one morphism to compose, just return that morphism
        if n == 1:
            return f_list[0]
        
        # Create representation
        rep = Repr_Composition(f_list)
        
        # Check if the representation already exists in the diagram, and if so, return the morphism it points to
        g = self.diagram.find_representation(rep)
        if g:
            return g

        # Construct new object/morphism
        g = Morphism(X.category, X, Y)
        
        # Assign, and add morphism and representation to the diagram
        rep.assign(g)
        self.diagram.add_morphism(g)
        self.diagram.add_representation(rep)
        
        return g
    
    def apply_functor(self, F, x):
        # x must belong to the domain category of the functor
        if(x.category != F.domain): # TODO: change this to a 'belongs to'-method (some trivial coercion might occur, e.g. R in Ring^op)
            raise Exception('Object/morphism does not belong to functor domain!')
        
        # Create representation
        rep = Repr_Functor(F, x)
        
        # Check if the representation already exists in the diagram, and if so, return the morphism it points to
        F_x = self.diagram.find_representation(rep)
        if F_x:
            return F_x
        
        # Construct new object/morphism
        if isinstance(x, Object):        
            F_x = Object(F.codomain)
        else:
            X = x.domain
            Y = x.codomain
            F_X = self.apply_functor(F, X)
            F_Y = self.apply_functor(F, Y)
            F_x = Morphism(F.codomain, F_X, F_Y)
        
        # Assign, and add morphism and representation to the diagram
        rep.assign(F_x)
        self.diagram.add_morphism(F_x)
        self.diagram.add_representation(rep)
        
        return F_x
    
    def apply_property(self, prop, data):
        # TODO: Check if the data satisfies the diagram of the property. If not, return Null / None / False, or something like that

        # Create representation
        rep = Repr_Property(prop, data)

        # Check if this representation already exists somewhere!
        C = self.diagram.find_representation(rep)
        if C:
            return C        
        
        # Construct new category
        C = Object(Cat)
    
        # Assign, and add object and representation to the diagram
        rep.assign(C)
        self.diagram.add_morphism(C)
        self.diagram.add_representation(rep)
    
        return C

### Core objects

In [11]:
Cat = Object()

_0 = Object(Cat)
_1 = Object(Cat)

### Examples

In [21]:
global_diagram = Diagram()
global_factory = Factory(global_diagram)

# Define in global diagram
Ring_op = global_factory.create_object('Ring\'', Cat) # Ring' : Cat
Scheme = global_factory.create_object('Scheme', Cat) # Scheme : Cat

Spec = global_factory.create_morphism('Spec', Ring_op, Scheme) # Spec : Ring' -> Scheme
Mod = global_factory.create_morphism('Mod', Ring_op, Cat) # Mod : Ring' -> Cat

R = global_factory.create_object('R', Ring_op) # R : Ring'
R_Mod = global_factory.apply_functor(Mod, R) # Mod(R)

# Property affine : diagram (Actually, lets say one needs to define the diagram after saying what the properties name is? That prevents someone from altering a diagram after it is being used. Think about the syntax in that case.)
diagram = Diagram()
factory = Factory(diagram)
X = factory.create_object('X', Scheme)
affine = Property('affine', diagram)

diagram = Diagram()
factory = Factory(diagram)

X = factory.create_object('X', Scheme) # X : Scheme
Y = factory.create_object('Y', Scheme) # Y : Scheme
f = factory.create_morphism('f', X, Y) # f : X -> Y

X_is_affine = factory.apply_property(affine, [ X ]) # affine(X)


In [22]:
str(global_diagram)

"{ Ring', Scheme, Spec, Mod, R, Mod(R) }"

In [23]:
str(diagram)

'{ X, Y, f, affine(X) }'

In [25]:
diagram = Diagram()
factory = Factory(diagram)

A = factory.create_object('A', Ring_op)
B = factory.create_object('B', Ring_op)
C = factory.create_object('C', Ring_op)
f = factory.create_morphism('f', A, B)
g = factory.create_morphism('g', B, C)
h = factory.create_morphism('h', B, B)

gf = factory.create_composition([ g, h, f ])

Spec_f = factory.apply_functor(Spec, f)

In [26]:
str(diagram)

'{ A, B, C, f, g, h, g.h.f, ?(A), ?(B), ?(f) }'